# Análise da Pesquisa sobre Falhas na Entrega de Rações

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from IPython.display import display, Markdown, Image
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
# Run the main script to process data and generate charts
!python ../main.py

## Introdução

Este relatório apresenta uma análise aprofundada das respostas coletadas através do formulário sobre falhas na entrega de rações. O objetivo é compreender os principais problemas enfrentados pelos diferentes atores da cadeia produtiva, identificar os sentimentos predominantes e propor ações de melhoria para otimizar o processo logístico.

## Metodologia

A análise foi realizada em três etapas principais:

1.  **Segmentação por Perfil:** As respostas foram agrupadas de acordo com o perfil do respondente (Produtor, Veterinário/Extensionista, Motorista, Gestão, etc.).
2.  **Análise de Sentimento e Padrões:** As respostas de cada grupo foram examinadas para identificar os principais problemas, a frequência com que ocorrem e o sentimento geral (negativo, neutro, positivo).
3.  **Elaboração do Diagnóstico e Recomendações:** Com base nos padrões identificados, foi elaborado um diagnóstico consolidado e um conjunto de ações de melhoria.

## Visualização dos Dados

In [ ]:
display(Markdown('### Frequência de Problemas por Perfil de Ator'))
display(Image(filename='../plots/problemas_por_perfil.png'))
display(Markdown('### Frequência de Sugestões de Melhoria por Perfil de Ator'))
display(Image(filename='../plots/sugestoes_por_perfil.png'))
display(Markdown('### Análise de Pareto dos Problemas (Agregado)'))
display(Image(filename='../plots/pareto_problemas_agregado.png'))

## Análise Estratégica

In [ ]:
with open('../docs/strategic_analysis_output.txt', 'r', encoding='utf-8') as f:
    strategic_analysis = f.read()
display(Markdown(strategic_analysis))

## Análise de Tópicos com NLP

In [ ]:
def get_topics(data, n_topics=5, n_top_words=5):
    stop_words = stopwords.words('portuguese')
    vectorizer = CountVectorizer(stop_words=stop_words)
    X = vectorizer.fit_transform(data)
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(X)
    feature_names = vectorizer.get_feature_names_out()
    topics = []
    for topic_idx, topic in enumerate(lda.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topics.append(f'Tópico {topic_idx + 1}: {.join(top_words)}')
    return topics

with open('../docs/analysis_by_profile.txt', 'r', encoding='utf-8') as f:
    content = f.read()

responses = re.findall(r'- Resposta \d+: (.*)', content)

topics = get_topics(responses)
display(Markdown('### Principais Tópicos Mencionados nas Respostas'))
for topic in topics:
    display(Markdown(f'- {topic}'))